In [8]:
import pandas as pd
import numpy as np
import os
import math
import warnings
import gurobipy as gp
from gurobipy import GRB


warnings.filterwarnings('ignore')
pd.options.display.max_rows = 9999


- aisle interval= 50
- aisle length = 500   
   
- batch size = 25
- Number of batch = 24 

In [2]:
# file_list=os.listdir('./dataset/Orderlinelist')
# for f in file_list:
#     orderlinelist=pd.read_csv('./dataset/Orderlinelist/'+f,delimiter='\t',header=None,engine='python')
#     orderlinelist.columns=orderlinelist_column
#     if orderlinelist['LocationID'].max()>2881:
#         print(f)
    

In [9]:
path='./dataset'

dist_matrix=pd.read_csv(path+'/Distance matrix/DistanceMatrix_24SubAisles.txt',delimiter='\t',header=None,engine='python')
dist_matrix=dist_matrix.loc[:,:2880]

#orderlist
orderlist_clolumn=['OrderID', 'NumberOfOrderLines', 'ReleaseTime', 'DueTime' ,'FirstOrderLineID']

#orderline
orderlinelist_column=['OrderID', 'OrderLineID', 'AisleID' ,'CellID', 'LevelID', 'LocationID']


orderlist=pd.read_csv(path+'/Orderlist/OrderList_LargeProblems_1_21.txt',delimiter='\t',header=None,engine='python')
orderlist.columns=orderlist_clolumn


orderlinelist=pd.read_csv(path+'/Orderlinelist/OrderLineList_LargeProblems_1_21.txt',delimiter='\t',header=None,engine='python')
orderlinelist.columns=orderlinelist_column



In [4]:
print('distance matrix size:',dist_matrix.to_numpy().shape)
print('orderlist size:','(',len(orderlist),',',len(orderlist.columns),')')
print('orderlinelist size:','(',len(orderlinelist),',',len(orderlinelist.columns),')',end='\n\n')

print('@ orderlist info @')
display(orderlist.info())
print('@ orderlinelist info @')
display(orderlinelist.info())


distance matrix size: (2881, 2881)
orderlist size: ( 595 , 5 )
orderlinelist size: ( 2484 , 6 )

@ orderlist info @
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 595 entries, 0 to 594
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype
---  ------              --------------  -----
 0   OrderID             595 non-null    int64
 1   NumberOfOrderLines  595 non-null    int64
 2   ReleaseTime         595 non-null    int64
 3   DueTime             595 non-null    int64
 4   FirstOrderLineID    595 non-null    int64
dtypes: int64(5)
memory usage: 23.4 KB


None

@ orderlinelist info @
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2484 entries, 0 to 2483
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   OrderID      2484 non-null   int64
 1   OrderLineID  2484 non-null   int64
 2   AisleID      2484 non-null   int64
 3   CellID       2484 non-null   int64
 4   LevelID      2484 non-null   int64
 5   LocationID   2484 non-null   int64
dtypes: int64(6)
memory usage: 116.6 KB


None

In [3]:
def random_batch(orderlist,orderlinelist,batch_size):
    orderlist_ix=orderlist.index
    NB=math.ceil(len(orderlist)/batch_size)

    for i in range(NB+1):
        if len(orderlist_ix) < batch_size:
            orderlinelist.fillna(i,inplace=True)
        else:
            orderlist_ix25=np.random.choice(orderlist_ix,batch_size)
            orderlist_ix=list(set(orderlist_ix)-set(orderlist_ix25))
        
            orderid_25=orderlist.loc[orderlist_ix25]['OrderID'].values
            orderlinelist.loc[orderlinelist['OrderID'].isin(orderid_25),'BatchID']=i
            
    return orderlinelist


def FCFS(orderlist,orderlinelist,batch_size):
    orderlinelist['BatchID']=None
    batch_size=25

    quotient=math.ceil(len(orderlist)/batch_size)

    for i in range(quotient+1):
        if len(orderlinelist[orderlinelist['BatchID'].isnull()]['OrderID'].unique()) < batch_size:
            orderlinelist.fillna(i,inplace=True)
        else:
            orderid_25=list(range(i*batch_size,(i+1)*batch_size))
            orderlinelist.loc[orderlinelist['OrderID'].isin(orderid_25),'BatchID']=i
    return orderlinelist



def distance(aisle_last,max_aisle):
    return (2*50*aisle_last + 500*(max_aisle+1) + (max_aisle-1)*2*50 + 500*max_aisle)/2

def routing_dist(orderlist,orderlinelist,batch_size):
    NB=math.ceil(len(orderlist)/batch_size)
    dist=0
    for i in range(NB+1):
        aisle_last=orderlinelist[orderlinelist['BatchID']==0]['AisleID'].max()
        max_aisle=len(orderlinelist[orderlinelist['BatchID']==0]['AisleID'].unique())
        dist+=distance(aisle_last,max_aisle)
    return dist

# Random & FCFS

In [107]:
orderlinelist['BatchID']=None
batch_size=25

#random_batch, FCFS
orderlinelist=random_batch(orderlist,orderlinelist,batch_size)
total_dist=routing_dist(orderlist,orderlinelist)

351250.0
351250.0
365000.0
351250.0
365000.0
365000.0
365000.0
365000.0
351250.0
365000.0
365000.0
351250.0
365000.0
365000.0
351250.0
365000.0
351250.0
351250.0
351250.0
365000.0
365000.0
365000.0
337500.0
337500.0
351250.0
337500.0
365000.0
365000.0
365000.0
365000.0
351250.0
365000.0
365000.0
365000.0
365000.0
365000.0
365000.0
365000.0
365000.0
365000.0
365000.0
365000.0
365000.0
351250.0
351250.0
351250.0
365000.0
365000.0
365000.0
365000.0
365000.0
365000.0
365000.0
365000.0
351250.0
365000.0
365000.0
365000.0
365000.0
351250.0
351250.0
365000.0
365000.0
365000.0
365000.0
365000.0
365000.0
365000.0
365000.0
365000.0
351250.0
365000.0
365000.0
365000.0
365000.0
351250.0
365000.0
337500.0
365000.0
365000.0
365000.0
337500.0
365000.0
365000.0
365000.0
365000.0
351250.0
365000.0
365000.0
365000.0
337500.0
365000.0
365000.0
365000.0
351250.0
337500.0
365000.0
365000.0
351250.0
365000.0


# Order Batching Heuristic

# MIP

In [ ]:
import gurobipy as gp
from gurobipy import GRB

# Variation of mip1.py, with a focus on remote services
#
# When remote resources are tied to the optimization process, such as a token
# server, compute server, or Instant Cloud, extra care should be taken to
# ensure that such resources are released once they are no longer needed.
# Technically, such resources are managed by a gurobipy.Env object
# ("environment").  This example shows best practices for acquiring and
# releasing such shared resources via Env objects.
#
# See also https://www.gurobi.com/documentation/9.1/refman/environments.html


def populate_and_solve(m):
    # This function formulates and solves the following MIP model (see mip1.py):
    #  maximize
    #        x +   y + 2 z
    #  subject to
    #        x + 2 y + 3 z <= 4
    #        x +   y       >= 1
    #        x, y, z binary

    # Create variables
    aisle_length=500
    aisle_width=50

    #number of orders
    order_n=len(list(orderlist.index))
    #order list index
    i=range(0,order_n)
    #batch index=24, 
    batch_size=25
    #number of batches
    batch=math.ceil(order_n/25)
    #batch index
    k=range(0,batch)
    #aisle index
    m=sorted(list(orderlinelist.AisleID.unique()))
    #number of aisles
    s=len(m)
    #A_mi
    A=[[aisle_number],[order_index]]



    x=m.addVar(vtype=GRB.BINARY, name='x')
    y=m.addVar(vtype=GRB.BINARY, name='y')
    la=m.addVar(vtype=GRB.INTEGER, name='la')
    N=m.addVar(vtype=GRB.INTEGER, name='N')
  
    # Set objective
    m.setObjective((((la-1)*2*aisle_width + aisle_length*N + (aisle_length*2*aisle_width) + aisle_length*(N+1)/2)), GRB.MINIMIZE)
    
    # Add constraint: x + 2 y + 3 z <= 4
    const1 = m.addConstrs((x.sum(i,'*') <= batch_size for i in order_n), name='const1')
    const2 = m.addConstrs((x.sum('*',k) == 1 for k in batch), name='const2')
    const3 = m.addConstrs(y=max(A[][] for *x[][]), name='const3')
    const4 = m.addConstrs(, name='const4')
    const5 = m.addConstrs(la<=s, name='const5')
    const6 = m.addConstrs(, name='const6')
    const7 = m.addConstrs(, name='const7')
    
    # Optimize model
    m.optimize()

    for v in m.getVars():
        print(f"{v.VarName} {v.X:g}")

    print(f"Obj: {m.ObjVal:g}")


# Put any connection parameters for Gurobi Compute Server, Gurobi Cluster
# Manager or Gurobi Token server here, unless they are set already
# through the license file.

connection_params = {
    # For Compute Server you need at least this
    #       "ComputeServer": "<server name>",
    #       "UserName": "<user name>",
    #       "ServerPassword": "<password>",
    # For Cluster Manager you need at least this
    #       "CSManager": "<manager name>",
    #       "CSAPIAccessID": "<access ID>",
    #       "CSAPISecret": "<secret>",
    # For Instant cloud you need at least this
    #       "CloudAccessID": "<access id>",
    #       "CloudSecretKey": "<secret>",
}

with gp.Env(params=connection_params) as env:
    # 'env' is now set up according to the connection parameters.
    # The environment is disposed of automatically through the context manager
    # upon leaving this block.
    with gp.Model(env=env) as model:
        # 'model' is now an instance tied to the enclosing Env object 'env'.
        # The model is disposed of automatically through the context manager
        # upon leaving this block.
        try:
            populate_and_solve(model)
        except:
            # Add appropriate error handling here.
            raise